# ADS Project 3 Group 4

## Libraries and Settings

In [4]:
from scipy.io import loadmat
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
from PIL import Image

E:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
"""
Path
"""
DATA_PATH = "../data/train_set"
IMAGE_FOLDER = os.path.join(DATA_PATH, "images")
POINTS_FOLDER = os.path.join(DATA_PATH, "points")
LABELS_FOLDER = DATA_PATH

## Read Training Data

In [6]:
def read_all_images():
    """
    Read 2500 training images from the IMAGE_FOLDER
    :return a 4d numpy array in form of (index, height, width, channels), channels is RGB 
    """
    files = [file for file in os.listdir(IMAGE_FOLDER) if file.endswith('.jpg')]
    files.sort()
    
    face_images = np.zeros((len(files), 750, 1000, 3))
    
    for index, filename in enumerate(files):
        face_img_arr = plt.imread(os.path.join(IMAGE_FOLDER, filename))
        if face_img_arr.shape != (750,1000,3):
            # resize the image
            face_img = Image.fromarray(face_img_arr)
            face_img = face_img.resize((1000,750))
            face_img_arr = np.array(face_img)
        face_images[index] = face_img_arr
    return face_images

def read_labels():
    """
    Read the image labels from the label.csv file
    :return a pandas.DataFrame with 3 columns: 'emotion_idx','emotion_cat','type'
    """
    labels_df = pd.read_csv(os.path.join(LABELS_FOLDER, 'label.csv'))
    labels_df = labels_df.loc[:,['emotion_idx','emotion_cat','type']]
    return labels_df
    

def read_all_points():
    """
    Read all face coordinates points
    :return a tuple of shape (2500, 78, 2). Because for each of 2500 images there are 78 points associated with it
    """
    files = [file for file in os.listdir(POINTS_FOLDER) if file.endswith('.mat')]
    files.sort()
    
    face_points = np.zeros((len(files), 78, 2))
    for index, filename in enumerate(files):
        face_points_dict = loadmat(os.path.join(POINTS_FOLDER, filename))
    
        face_points[index] = face_points_dict.get('faceCoordinatesUnwarped',  face_points_dict.get('faceCoordinates2'))
    return face_points

def load_data(loadImage = False):
    """
    Load training data from local files
    
    :loadImage if it's False, this function will not load original images
    :return a tuple (images, points, labels)
        if loadImage is False, the 'images' will None. Otherwise its a numpy array with shape (2500,750,1000,3)
        points is a numpy array with shape (2500, 78, 2)
        labels is a pandas.DataFrame
    """
    #face_images_narr =  read_all_images() if loadImage else None
    face_images_points = read_all_points()
    labels = read_labels()
    #return face_images_narr, face_images_points, labels
    return face_images_points, labels

In [7]:
# pass True if you want to read original images, it might take some time to do it
#images, points, labels = load_data(True)
points, labels = load_data(True)

In [5]:
#if images:
#    print(images.shape)

print(points.shape)
labels

(2500, 78, 2)


,emotion_idx,emotion_cat,type
0,1,Neutral,simple
1,1,Neutral,simple
2,1,Neutral,simple
3,1,Neutral,simple
4,1,Neutral,simple
...,...,...,...
2495,22,Sadly disgusted,compound
2496,22,Sadly disgusted,compound
2497,22,Sadly disgusted,compound
2498,22,Sadly disgusted,compound


## Feature Selection

In [8]:
from sklearn.metrics import pairwise_distances

def feature(input_points):
    n = input_points.shape[0]
    pairwise_dist_data = []
    # return a vector
    def pairwise_dist(vec):
        vec = np.reshape(vec, (len(vec),1))
        dist_matrix = pairwise_distances(vec)
        dist_matrix = dist_matrix[np.triu_indices(dist_matrix.shape[0], k=1)]
        return dist_matrix
    
    # dist is an 2 column array
    def pairwise_dist_result(mat):
        dist = np.apply_along_axis(func1d=pairwise_dist, axis=0, arr=mat)
        dist_result = np.ndarray.flatten(dist,order='F').tolist()
        return dist_result
    
    for i in range(n):
        pairwise_dist_feature = pairwise_dist_result(points[i,:,:])
        pairwise_dist_data.append(pairwise_dist_feature)
        
    pairwise_dist_data = pd.DataFrame(pairwise_dist_data)
    return pairwise_dist_data
        


In [9]:
X = feature(points)
y = labels['emotion_idx']

## Model Traning and Selection

In [10]:
from sklearn.model_selection import train_test_split
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

E:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [10]:
# model selection with cross-validation 
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
para1 = {
    'n_estimators':[100,200,300,400,500,600]  
}
xgb_model = xgb.XGBClassifier(learning_rate=0.1, max_depth=1)
gsearch1 = GridSearchCV(estimator = xgb_model, 
                        param_grid = para1, 
                        scoring ='accuracy',
                        cv = 5,
                        n_jobs = 4
                       )
gsearch1.fit(X_train,y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=5, verbose=False)

E:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=1, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=4,
             param_grid={'n_estimators': [100, 200, 300, 400, 500, 600]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring

In [12]:
gsearch1.best_params_, gsearch1.best_score_

({'n_estimators': 100}, 0.363)

In [14]:
para2 = {
    'max_depth': [1,2,3]
}
xgb_model2 = xgb.XGBClassifier(learning_rate=0.1, n_estimators=100)
gsearch2 = GridSearchCV(estimator = xgb_model2, 
                        param_grid = para2, 
                        scoring ='accuracy',
                        cv = 5,
                        n_jobs = 4
                       )
gsearch2.fit(X_train,y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=5, verbose=False)

E:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=4, param_grid={'max_depth': [1, 2, 3]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [16]:
gsearch2.best_params_, gsearch2.best_score_

({'max_depth': 3}, 0.42)

In [17]:
para3 = {
    'learning_rate': [0.05,0.1]
}
xgb_model3 = xgb.XGBClassifier(max_depth=3 , n_estimators=100)
gsearch3 = GridSearchCV(estimator = xgb_model, 
                        param_grid = para3, 
                        scoring ='accuracy',
                        cv = 5,
                        n_jobs = 4
                       )
gsearch3.fit(X_train,y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)

E:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=1, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=4, param_grid={'learning_rate': [0.05, 0.1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [18]:
gsearch3.best_params_, gsearch3.best_score_

({'learning_rate': 0.1}, 0.363)

 - The best parameters: depth=3, learning_rate=0.1, number of estimators=100

## Testing

In [11]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
xgb_best1 = xgb.XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=100)
xgb_best1.fit(X_train, y_train)
pred1 = xgb_best1.predict(X_test)
print("accuracy: {}".format(accuracy_score(y_test, pred1)))

accuracy: 0.5
